In [12]:
import psycopg2
import time
import os
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv(".env")

USER = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(user=USER, password=PASSWORD, host="localhost", port="5432", dbname="app_auth")
cur = conn.cursor()

# Récupérer le nombre total d'utilisateurs dans user_table
cur.execute("SELECT COUNT(*) FROM user_table")
total_users = cur.fetchone()[0]

# Fonction pour mettre à jour les usernames avec trigger
def update_usernames_with_trigger(n):
    start_time = time.time()
    for i in range(1, min(n, total_users) + 1):
        new_username = f"user_trigger_{i:07d}"
        cur.execute("UPDATE user_table SET username = %s WHERE user_id = %s", (new_username, i))
    conn.commit()
    end_time = time.time()
    return end_time - start_time

# Fonction pour mettre à jour les usernames manuellement et enregistrer dans username_history_from_app
def update_usernames_manually(n):
    start_time = time.time()
    for i in range(1, min(n, total_users) + 1):
        new_username = f"user_manual_{i:07d}"
        cur.execute("UPDATE user_table SET username = %s WHERE user_id = %s", (new_username, i))
        cur.execute("INSERT INTO username_history_from_app (user_id, username_new) VALUES (%s, %s)", (i, new_username))
    conn.commit()
    end_time = time.time()
    return end_time - start_time

# Comparer les temps d'exécution pour 100, 1000 et 10000 mises à jour
times_with_trigger = {n: update_usernames_with_trigger(n) for n in [100, 1000, 10000]}
times_manually = {n: update_usernames_manually(n) for n in [100, 1000, 10000]}

# Afficher les résultats
print("Temps d'exécution avec Trigger:", times_with_trigger)
print("Temps d'exécution avec Mise à jour Manuelle:", times_manually)

# Fermer la connexion
cur.close()
conn.close()


ForeignKeyViolation: ERREUR:  une instruction insert ou update sur la table « username_history_from_app » viole la contrainte de clé
étrangère « username_history_from_app_user_id_fkey »
DETAIL:  La clé (user_id)=(101) n'est pas présente dans la table « user_table ».
